In [67]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import os
import json
import ast
from enum import Enum
from src import scrab_data
import csv

In [41]:
ALL_TICKERS_FOLDER = "./all_tickers"

In [61]:
class Action(Enum):
    BUY = 0
    SELL = 1

In [42]:
class Environment:
    _instance = None
    stocks_objects = list() # might be useless
    all_ticker_dfs = dict() # tickers with corresponding dataframes

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Environment, cls).__new__(cls)
            cls._instance._initialized = False
        return cls._instance

    def __init__(self):
        if not self._initialized:
            # Load csv files into memory
            files = os.listdir(ALL_TICKERS_FOLDER)
            for csv_file in tqdm(files, desc="Loading CSVs into memory", unit="files"):
                try:
                    df = pd.read_csv(
                        os.path.join(ALL_TICKERS_FOLDER, csv_file),
                        index_col="date",                     # index column name
                        parse_dates=["date"],                 # explicitly parse this column as datetime
                    )
                    df.index = df.index.normalize()           # remove any time component
                    ticker = csv_file.removesuffix(".csv")
                    self.all_ticker_dfs[ticker] = df
                except Exception as e:
                    print(f"Error reading {csv_file}: {e}")
                    raise RuntimeError
            # Only initialize once
            self._initialized = True

In [43]:
env = Environment()

Loading CSVs into memory: 100%|██████████| 2622/2622 [00:14<00:00, 181.66files/s]


In [44]:
class Tickers:
    df_tickers = pd.DataFrame()
    ticker_file_name = "tickers.csv"
    url = "https://app.scrab.com/screener/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0",
        "Referer": "https://app.scrab.com/screener/",
        "Origin": "https://app.scrab.com",
        "Hx-Request": "true",
        "X-CSRFToken": scrab_data.CSRF_TOKEN,
    }
    cookies = {
        "sessionid": scrab_data.SESSION_ID,
        "csrftoken": scrab_data.CSRF_TOKEN
    }

    def __init__(self):
        try:
            self.df_tickers = pd.read_csv(self.ticker_file_name)
            self.df_tickers.index = pd.to_datetime(self.df_tickers.index)
        except FileNotFoundError:
            self.fetch_tickers()
            self.df_tickers.to_csv(self.ticker_file_name, index=False)

    def fetch_tickers(self, debug=False):
        self.df_tickers = pd.DataFrame()
        i = 0

        current_rows = self.df_tickers.shape[0]
        fetch_more = True

        while fetch_more:
            fetch_more = False
            market_cap_less = None
            if current_rows > 0:
                self.df_tickers.sort_values(by="Market Cap (USD)", inplace=True, ascending=False, key=lambda x: x.astype(float))
                market_cap_less = str(float(self.df_tickers["Market Cap (USD)"].iloc[-1]) - 1)
                if debug:
                    print(f"Fetching more tickers with market cap less than {market_cap_less}")

            fetched_tickers_data = self.fetch_data(market_cap_less=market_cap_less)
            self.parse_data(fetched_tickers_data)

            if debug:
                self.df_tickers.to_csv(f"tickers_page_{i}.csv", index=False)

            if self.df_tickers.shape[0] > current_rows:
                fetch_more = True
                current_rows = self.df_tickers.shape[0]
                i += 1
                time.sleep(1)
            
            if current_rows % 10 != 0:
                self.df_tickers.sort_values(by="Market Cap (USD)", inplace=True, ascending=False, key=lambda x: x.astype(float))
                print(f"Fetched {current_rows} tickers, exit loop")
                fetch_more = False

    def parse_data(self, html_data):
        soup = BeautifulSoup(html_data, 'html.parser')
        table = soup.find("table", {"id": "screener-results-table"})
        headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")][1:]  # skip checkbox

        rows = []
        for tr in table.find("tbody").find_all("tr"):
            tds = tr.find_all("td")[1:]  # skip row number
            row = []
            for td in tds:
                a = td.find("a")
                if a:
                    row.append(a.get_text(strip=True))
                else:
                    span = td.find("span")
                    if span:
                        row.append(span.get_text(strip=True))
                    else:
                        row.append(td.get_text(strip=True))
            rows.append(row)

        new_tickers = pd.DataFrame(rows, columns=headers)
        new_tickers["Revenue Average Estimated Annual Growth in Next 2 Years"] = new_tickers["Revenue Average Estimated Annual Growth in Next 2 Years"].str.replace(r'[^0-9.]', '', regex=True)
        new_tickers["EPS Average Estimated Annual Growth in Next 2 Years"] = new_tickers["EPS Average Estimated Annual Growth in Next 2 Years"].str.replace(r'[^0-9.]', '', regex=True)
        new_tickers["EPS Long Term (5 Years) Growth Estimates"] = new_tickers["EPS Long Term (5 Years) Growth Estimates"].str.replace(r'[^0-9.]', '', regex=True)
        new_tickers["CFO per Share Average Estimated Annual Growth in Next 2 Years"] = new_tickers["CFO per Share Average Estimated Annual Growth in Next 2 Years"].str.replace(r'[^0-9.]', '', regex=True)

        self.df_tickers = pd.concat([self.df_tickers, new_tickers], ignore_index=True)

    def fetch_data(self, market_cap_less=None, debug=False):
        form_data = {
            "csrfmiddlewaretoken": scrab_data.CSRF_TOKEN,
            "countries": '[{"value":"USA","code":"us","extra":"AMEX, NASDAQ, NYSE","searchBy":"NasdaqGS,Global,Capital,New,York,North,America,United,States,AMEX,NASDAQ,NYSE"}]',
            "market_cap_greater": "10000000000",
            "sales_est_2fy_avg_greater": "5",
            "eps_est_2fy_avg_greater": "7",
            "price_greater": "15",
            "eps_est_2y_num_est_greater": "5",
            "eps_est_long_term_growth_num_est_greater": "5",
            "eps_est_long_term_growth_greater": "10",
            "cfo_per_share_est_2fy_avg_greater": "7",
        }
        if market_cap_less is not None:
            form_data["market_cap_less"] = market_cap_less

        response = requests.post(
            "https://app.scrab.com/screener/",
            data=form_data,
            headers=self.headers,
            cookies=self.cookies
        )

        if debug:
            print(response.status_code)
            with open("fetch_tickers_response.html", "w", encoding="utf-8") as f:
                f.write(response.text)
        return response.text

    def get_tickers(self):
        return self.df_tickers["Ticker"].to_list()

# Load or fetch tickers
# tickers = Tickers()

In [45]:
# screener = tickers.get_tickers()

In [46]:
class Stock:
    def __init__(self, ticker, fetch_from_scrab=False):
        env = Environment()
        self.expensiveness_coeff = 1.2

        self.ticker = ticker
        self.df = pd.DataFrame()
        self.trela_indicator = None

        if fetch_from_scrab:
            # Fetch Strategy metrics
            self.fetch_metric("price")
            self.fetch_metric("price_target")
            self.fetch_metric("price_target_upside")
            self.fetch_metric("forward_pe_ratio")
            self.fetch_metric("forward_pe_ratio:median:3")
            self.fetch_metric("forward_ps_ratio")    
            self.fetch_metric("forward_ps_ratio:median:3")
            # Fetch Screener metrics
            self.fetch_metric("market_cap")
            self.fetch_metric("sales_est_2fy_avg")
            self.fetch_metric("eps_est_2fy_avg")
            self.fetch_metric("eps_est_2y_num_est")
            self.fetch_metric("eps_est_long_term_growth_num_est")
            self.fetch_metric("eps_est_long_term_growth")
            self.fetch_metric("cfo_per_share_est_2fy_avg")
        else:
            self.df = env.all_ticker_dfs[ticker]

        # Placeholder metrics
        # Initialize buy / sell to false
        for col in ['buy', 'sell']:
            if col not in self.df.columns:
                self.df[col] = False
                
        self.postprocess_metrics()
        env.stocks_objects.append(self)

    def fetch_metric(self, metric):
        try:
            response_metric = self._request_data(self.ticker, metric)
            datapoints = response_metric[0]['datapoints']
            
            metric_df = pd.DataFrame(datapoints, columns=['timestamp', metric])
            metric_df.insert(0, 'date', pd.to_datetime(metric_df['timestamp'], unit='s'))
            metric_df = metric_df.drop(columns='timestamp')

        except (IndexError, KeyError, TypeError, ValueError) as e:
            # Handle missing or malformed data
            print(f"[Warning] Failed to fetch metric '{metric}' for {self.ticker}: {e}")

            # Insert zeros aligned with existing df dates, or just create a dummy df
            if self.df.empty:
                # No existing dates — create dummy one-row entry with today's date
                metric_df = pd.DataFrame({
                    'date': [pd.Timestamp.today()],
                    metric: [np.nan]
                })
            else:
                # Add column of 0s for existing dates
                metric_df = pd.DataFrame({
                    'date': self.df['date'],
                    metric: [np.nan] * len(self.df)
                })

        for col in self.df.columns:
            if col != 'date' and self.df[col].isna().all():
                print(f"[Warning] Column '{col}' is all NaN")


        # Merge the metric into self.df
        if self.df.empty:
            self.df = metric_df
        else:
            self.df = pd.merge(self.df, metric_df, on='date', how='left')


    def postprocess_metrics(self):
        # Forward fill price targets
        self.df['price_target'] = self.df['price_target'].ffill()
        self.get_trend("price_target", add_to_dataframe=True)
        # self.calculate_trela_indicator() // out of date, update!

    def _request_data(self, ticker, metric):
        url = "https://app.scrab.com/data/metric/"
        payload = {
        "tickers":[ticker],
        "metrics":[metric],
        "economic":[],
        "index":1
        }
        cookies = {
            "sessionid": scrab_data.SESSION_ID,
            "csrftoken": scrab_data.CSRF_TOKEN
        }
        headers = {
            "Content-Type": "application/json",
            "User-Agent": "Mozilla/5.0",
            "X-CSRFToken": scrab_data.CSRF_TOKEN,
            "Referer": "https://app.scrab.com/charts/master/",
            "Origin": "https://app.scrab.com"
        }
        response = requests.post(url, json=payload, headers=headers, cookies=cookies)
        data = response.json()
        return data
  
    def plot(self, title, metrics, log_scale=False, start_date="1900-01-01", save=False):
        df_plot = self.df.loc[start_date:]
        fig, ax = plt.subplots(figsize=(12,6))
        for metric in metrics:
            ax.plot(df_plot.index, df_plot[metric], label=metric)
            ax.set_title(title)
            ax.set_xlabel('Date')
            ax.set_ylabel(str(metrics))
            ax.legend()
            ax.grid(True)
            if log_scale:
                ax.set_yscale('log')  # make y-axis logarithmic
        if (save):
            charts_dir = "./charts"
            os.makedirs(charts_dir, exist_ok=True)  # ensure folder exists
            chart_path = os.path.join(charts_dir, f"{self.ticker}.png")
            fig.savefig(chart_path, bbox_inches="tight")
            plt.close(fig)  # close to free memory if plotting multiple charts
        return ax

    def calculate_rolling_sigma_bands(self, metric, window_upper, window_lower, sigma_coeff = 2.0, smooth_window=1):
        min_periods = max(window_upper, window_lower) 
        # rolling mean & std
        mean_upper = self.df[metric].rolling(window_upper, min_periods=window_upper).mean()
        std_upper = self.df[metric].rolling(window_upper, min_periods=window_upper).std()
        mean_lower = self.df[metric].rolling(window_lower, min_periods=window_lower).mean()
        std_lower = self.df[metric].rolling(window_lower, min_periods=window_lower).std()
        # bands
        self.df[metric + '_upper_band'] = mean_upper + sigma_coeff * std_upper
        self.df[metric + '_lower_band'] = mean_lower - sigma_coeff * std_lower
        # smooth
        self.df[metric + '_upper_band'] = self.df[metric + '_upper_band'].rolling(smooth_window, min_periods=1).mean()
        self.df[metric + '_lower_band'] = self.df[metric + '_lower_band'].rolling(smooth_window, min_periods=1).mean()
        # drop unpopulated rows
        # self.df.dropna(subset=[metric + '_upper_band', metric + '_lower_band'], inplace=True)
    
    def get_sigma(self, metric, window=252):
        sigma_coeff = 1.0
        mean = self.df[metric].tail(window).mean()
        std = self.df[metric].tail(window).std()
        upper_band = mean + sigma_coeff * std
        lower_band = mean - sigma_coeff * std
        return mean, std, upper_band, lower_band

    def get_trend(self, metric, window=252, add_to_dataframe=False, debug=False):
        # TODO do this for the entire df, not only last value
        series = self.df[metric].tail(window)
        # Drop NaNs
        y = series.dropna().to_numpy()
        if len(y) < 2:
            if debug:
                print(f"[Warning] Not enough data to calculate trend for '{metric}' (only {len(y)} points).")
            if add_to_dataframe:
                self.df.loc[self.df.index[-window:], metric + '_trend'] = np.nan
            return
        X = np.arange(len(y)).reshape(-1, 1)
        model = LinearRegression().fit(X, y)
        
        # create trend line points so it can be plotted
        # and add it to the dataframe
        if add_to_dataframe:
            trend_data_points = model.predict(X)
            trend_series = pd.Series(np.nan, index=series.index)
            trend_series.iloc[-len(y):] = trend_data_points
            self.df.loc[self.df.index[-window:], metric + '_trend'] = trend_series

        if debug:
            print(model.coef_)
            print(model.intercept_)
        return model.coef_[0] > 0
    
    # to calculate Trela Indicator we need to calculate in order:
    # 1. Is the stock expensive? - forward_pe_ratio > 1.2 * forward_pe_ratio:median:3
    #                              forward_ps_ratio > 1.2 * forward_ps_ratio:median:3
    # FOR NOW WE WILL CHECK IF forward_pe_ratio > than forward_pe_ratio + sigma_coeff
    # 2. Is price target upside very low? - price_target_upside < lower_band (1.75 sigma)
    # 3. Is price target trend positive? - get_trend("price_target", window=252) > 0
    def calculate_trela_indicator(self, debug=False):
        # TODO do this for the entire df, not only last value
        latest_pe_ratio = self.df['forward_pe_ratio'].iloc[-1]
        latest_pe_ratio_median = self.df['forward_pe_ratio:median:3'].iloc[-1]
        latest_ps_ratio = self.df['forward_ps_ratio'].iloc[-1]
        latest_ps_ratio_median = self.df['forward_ps_ratio:median:3'].iloc[-1]

        if pd.isna(latest_pe_ratio) or pd.isna(latest_pe_ratio_median):
            print(f"[Warning] Skipping Trela indicator for {self.ticker} due to missing PE ratio data.")
            return

        is_very_expensive = False
        if latest_pe_ratio > latest_pe_ratio_median * self.expensiveness_coeff\
        or latest_ps_ratio > latest_ps_ratio_median * self.expensiveness_coeff:
            is_very_expensive = True
        
        if debug:
            print(f"{self.ticker} - latest_pe_ratio: {latest_pe_ratio}, "
                  f"latest_pe_ratio_median: {latest_pe_ratio_median}, "
                  f"latest_ps_ratio: {latest_ps_ratio}, "
                  f"latest_ps_ratio_median: {latest_ps_ratio_median}, "
                  f"expensiveness_coeff: {self.expensiveness_coeff}, "
                  f"is_very_expensive: {is_very_expensive}"
            )
        
        latest_upside_lower_band = self.get_sigma("price_target_upside", window=126)[-1]
        latest_upside = self.df['price_target_upside'].iloc[-1]
        is_upside_very_low = latest_upside < latest_upside_lower_band

        is_price_target_trend_positive = self.get_trend("price_target", window=252)

        if debug:
            print(f"{self.ticker} - latest_upside: {latest_upside}, "
                  f"latest_upside_lower_band: {latest_upside_lower_band}, "
                  f"is_upside_very_low: {is_upside_very_low}, "
                  f"is_price_target_trend_positive: {is_price_target_trend_positive}"
            )

        if is_very_expensive:
            if is_upside_very_low:
                self.trela_indicator = "SELL"
            else:
                self.trela_indicator = "HOLD"
        else:
            if is_upside_very_low:
                if is_price_target_trend_positive:
                    self.trela_indicator = "HOLD"
                else:
                    self.trela_indicator = "SELL"
            else:
                self.trela_indicator = "HOLD"

        if debug:
            print(f"{self.ticker} - Trela Indicator: {self.trela_indicator}")

In [47]:
# all_tickers = list()
# all_tickers_path = "all_tickers.json"
# with open(all_tickers_path, "r") as f:
#     companies = json.load(f)
# all_tickers = [entry["ticker"] for entry in companies]
# if os.path.isdir("./all_tickers_raw"):
#         print("Folder exists!")
# else:
#     os.makedirs("all_tickers_raw")

In [48]:
# THIS TAKES A LOT OF TIME:
# for ticker in tickers:
#     if os.path.isfile("./all_tickers_raw/" + ticker + ".csv"):
#         continue
#     print(f"fetching {ticker}")
#     current_ticker = Stock(ticker, for_screening=True)
#     current_ticker.df.to_csv(f"all_tickers_raw/{ticker}.csv")


In [49]:
# skipped_tickers = list()
# for ticker in tqdm(all_tickers, desc="Processing tickers"):
#     try:
#         df = pd.read_csv(f"./all_tickers_raw/{ticker}.csv")
#         # Drop useless columns:
#         df.drop(['buy', 'sell', 'price_target_trend'], axis=1, inplace=True)
#         df.ffill(inplace=True)

#         first_valid_idx = df.dropna().first_valid_index()
#         if first_valid_idx is None:
#             skipped_tickers.append(ticker)
#             continue
#         df_trimmed = df.loc[first_valid_idx:].reset_index(drop=True)

#         if not os.path.isdir("./all_tickers"):
#             os.makedirs("all_tickers")
            
#         df_trimmed.to_csv(f"all_tickers/{ticker}.csv", index=False)
#     except Exception as e:
#         tqdm.write(f"[{ticker}] ❌ Error: {e}")

In [50]:
# for ticker in all_tickers:
#     try:
#         df = pd.read_csv(f"./all_tickers/{ticker}.csv")
#         df.drop(df.columns[0], axis=1, inplace=True)
#         df.to_csv(f"all_tickers/{ticker}.csv", index=False)
#     except:
#         pass

In [51]:
# for ticker in screener:
#     stock = Stock(ticker)
#     latest_upside = stock.df['price_target_upside'].iloc[-1]
#     latest_upper_band = stock.df['price_target_upside_upper_band'].iloc[-1]
#     latest_lower_band = stock.df['price_target_upside_lower_band'].iloc[-1]
#     if latest_upside >= latest_upper_band:
#         print(f"{stock.ticker}: BUY -- Upside {latest_upside:.2f}, {latest_upside-latest_upper_band:.1f}% better than 2 sigma. Trela indicator: {stock.trela_indicator}")
#         ax = stock.plot(
#         stock.ticker,
#         ["price_target_upside", "price_target_upside_upper_band", "price_target_upside_lower_band"],
#         start_date="2020-01-01"
#         )
#     if latest_upside <= latest_lower_band:
#         print(f"{stock.ticker}: SELL -- Upside {latest_upside:.2f}, {latest_lower_band-latest_upside:.1f}% worse than 2 sigma. Trela indicator: {stock.trela_indicator}")
#         ax = stock.plot(
#         stock.ticker,
#         ["price_target_upside", "price_target_upside_upper_band", "price_target_upside_lower_band"],
#         start_date="2020-01-01"
#         )

In [52]:

class Screener:
    def __init__(self):
        pass
            
    def run(self, target_date):
        screener_tickers = list()
        for ticker, df in self.env.all_ticker_dfs.items():
            try:
                day_data = df.loc[target_date]
            except:
                continue
            row = day_data  # The single row for that date
            year = pd.to_datetime(target_date).year
            conditions = [
                # Adjusted marketcap (10B in 2025)
                row["market_cap"] >= self._adjust_market_cap_threshold(year),

                # Revenue Average Estimated Annual Growth in 2 Years > 5%
                row["sales_est_2fy_avg"] > 5,

                # EPS Average Estimated Annual Growth in Next 2 Years > 7%
                row["eps_est_2y_num_est"] > 7,

                # Price > 5
                row["price"] > 5,

                # EPS Number Estimates for 2 Fiscal Years Ahead > 5
                row["eps_est_2fy_avg"] > 5,

                # EPS Long Term (5 Years) Growth Number Estimates > 5
                row["eps_est_long_term_growth_num_est"] > 5,

                # EPS Long Term (5 Years) Growth Estimates > 10%
                row["eps_est_long_term_growth"] > 10,

                # CFO per Share Average Estimated Annual Growth in Next 2 Years > 7%
                row["cfo_per_share_est_2fy_avg"] > 7,
            ]
            if all(conditions):
                screener_tickers.append(ticker)
            else:
                pass
        return screener_tickers

    def _adjust_market_cap_threshold(self, target_year, base_threshold=10e9, base_year=2025):
        sp500_market_cap = {
            2000: 15,
            2001: 13,
            2002: 11,
            2003: 14,
            2004: 16,
            2005: 17,
            2006: 19,
            2007: 19,
            2008: 11,
            2009: 15,
            2010: 17,
            2011: 15,
            2012: 18,
            2013: 14,
            2014: 18,
            2015: 17,
            2016: 19,
            2017: 22,
            2018: 21,
            2019: 26,
            2020: 31,
            2021: 40,
            2022: 32,
            2023: 40,
            2024: 49,
            2025: 57,    # Unsure if this is 100% correct but doesn't really matter
        }
        
        if sp500_market_cap is None:
            raise ValueError("Please provide a market cap dictionary")

        if target_year not in sp500_market_cap or base_year not in sp500_market_cap:
            raise ValueError(f"Missing data for year {target_year} or {base_year}")

        ratio = sp500_market_cap[target_year] / sp500_market_cap[base_year]
        return base_threshold * ratio

In [ ]:
class Strategy:
    def __init__(self, ticker, date, buy_rule, sell_rule):
        self.stock = Stock(ticker)
        self.date = pd.to_datetime(date).normalize()
        self.prepare_metrics()
        self.buy_rule = buy_rule    # function: row -> bool
        self.sell_rule = sell_rule  # function: row -> bool

    def prepare_metrics(self):
        self.stock.calculate_rolling_sigma_bands("price_target_upside", 504, 252, 2, 504)
        # self.stock.plot(self.stock.ticker, ["price_target_upside", "price_target_upside_upper_band", "price_target_upside_lower_band"])
        pass

    def should_buy(self, row):
        return self.buy_rule(row)

    def should_sell(self, row):
        return self.sell_rule(row)
    
    def execute(self):
        df = self.stock.df
        # Try to get the row with given date, on KeyError do nothing.
        # Some manually adjusted csvs will throw an exception here, after testing it looks OK
        try:
            row = df.loc[self.date]
        except KeyError:
            return
        if self.should_buy(row):
            return Action.BUY
        elif self.should_sell(row):
            return Action.SELL
        

In [ ]:
class Position:
    def __init__(self, ticker):
        self.ticker = ticker
        self.size = 0
    
    def open(self, ticker, size, entry_date, entry_price):
        self.size += size
        
    def close(self, ticker, size, exit_date, exit_price):


In [ ]:
class Portfolio:
    def __init__(self):
        self.balance = 0
        self.open_positons = list(Position)

In [54]:
class Scorer:
    def __init__(self, stocks, criteria):
        self.stocks = stocks
        # What criteria?
        self.criteria = criteria

In [55]:
class AssetAllocator:
    def __init__(self, stocks, allocation):
        self.stocks = stocks
        self.stock_to_allocation = dict()
        allowed_allocation = set('equal_weights')
        if allocation not in allowed_allocation:
            raise ValueError(f'allocation must be one of {allowed_allocation}, got {allocation}')
        self.allocation = allocation

        match self.allocation:
            case 'equal_weights':
                n = len(self.stocks)
                self.stock_to_allocation = {stock: 1 / n for stock in stocks}


In [ ]:
class Backtester:
    def __init__(self, strategy, start_date = "2010-01-01", save_strategy=False):
      self.strategy = strategy
      self.simulation_date = pd.to_datetime(start_date)
      self.save_strategy = save_strategy
      self.last_date = pd.to_datetime("2025-10-11") # Last date from downloaded tickers
      self.screener_df = pd.DataFrame()
      self.date_to_screened_tickers = dict()
      self.screener_out_path = "./screener_out.csv"
      self.run_screener = False
      if os.path.isfile(self.screener_out_path):
        self.screener_df = pd.read_csv(self.screener_out_path)
        self.screener_df['date'] = pd.to_datetime(self.screener_df['date'])
        self.screener_df['tickers'] = self.screener_df['tickers'].apply(ast.literal_eval)
        self.date_to_screened_tickers = dict(zip(self.screener_df['date'], self.screener_df['tickers']))
      else:
        self.run_screener = True
        self.screener = Screener()
      if self.save_strategy:
        self.f_strategy = open("strategy.csv", mode="w", newline="", encoding="utf-8")
        self.strategy_writer = csv.writer(self.f_strategy)
        self.strategy_writer.writerow(["date", "buy_list", "sell_list"])

    def __del__(self):
      self.f_strategy.close()

    def run(self):
      # 1. Run Screener
      total_days = (self.last_date - self.simulation_date).days
      if self.run_screener:
        for _ in tqdm(range(total_days), desc="Screening", unit="days"):
          screener_tickers = self.screener.run(self.simulation_date)
          if screener_tickers:
            self.date_to_screened_tickers[self.simulation_date] = screener_tickers
          self.simulation_date += pd.Timedelta(days=1)
        # Neatly save to .csv
        self.screener_df = pd.DataFrame([{'date': date, 'tickers': tickers} for date, tickers in self.date_to_screened_tickers.items()])
        self.screener_df['date'] = self.screener_df['date'].astype(str)
        self.screener_df.to_csv(self.screener_out_path, index=False)

      # 2. Run Simulation
      for date, ticker_list in tqdm(self.date_to_screened_tickers.items(), total=len(self.date_to_screened_tickers), desc="Running simulation", unit="days"):
        # Strategy
        buy_rule = lambda row: row['price_target_upside'] > row['price_target_upside_upper_band']
        sell_rule = lambda row: row['price_target_upside'] < row['price_target_upside_lower_band']
        buy_list = list()
        sell_list = list()
        # 4. Run Strategy
        for ticker in ticker_list:
          strategy = Strategy(ticker, date, buy_rule, sell_rule)
          action = strategy.execute()
          if action is Action.BUY:
            buy_list.append(ticker)
          elif action is Action.SELL:
            sell_list.append(ticker)
        if self.save_strategy:
          self.strategy_writer.writerow([date, "|".join(buy_list), "|".join(sell_list)])
          self.f_strategy.flush()        
      # 5. Run Scorer
      # 6. Run Portioner
      # 7. Simulate buy/sell
      # 8. Calculate portfolio metrics



        
    # def run(self):
        # Inital pass for portioning. We don't know 
        # for stock in self.stocks:
        #     last_trading_day = stock.df['date'].max()
        #     today = pd.Timestamp.today().normalize()
        #     stop_date = min(last_trading_day, today)
            
        #     print(f'Simulation time: {(stop_date - self.simulation_date).days} days')

        #     while self.simulation_date <= stop_date:
                # Sometime start date is non-trading day, find first date that matches the start date
                # df_slice = stock.df[stock.df['date'] <= self.simulation_date]
                # latest_date = df_slice.iloc[-1]['date']
                # latest_row = df_slice.iloc[-1]
                # print(self.simulation_date)
                # Determine which stocks to buy
                # buy = strategy.should_buy(latest_row)
                # sell = strategy.should_sell(latest_row)
                # assert not (buy and sell)
                # if buy:
                #     stock.df.loc[stock.df['date'] == latest_date, 'buy'] = buy
                # elif sell:
                #     stock.df.loc[stock.df['date'] == latest_date, 'sell'] = sell

                # self.simulation_date += pd.Timedelta(days=1)

                # Decide stock allocation
                


allocation = 'equal_weights'
# strategy = Strategy(buy_rule, sell_rule)
backtester = Backtester(None)
backtester.run()

Running simulation:   0%|          | 2/3968 [00:00<04:44, 13.93days/s]

No data available for BAX on 2010-01-04 00:00:00
No data available for BAX on 2010-01-05 00:00:00
No data available for BAX on 2010-01-06 00:00:00


Running simulation:   0%|          | 4/3968 [00:00<04:44, 13.96days/s]

No data available for BAX on 2010-01-07 00:00:00
No data available for BAX on 2010-01-08 00:00:00
No data available for BAX on 2010-01-11 00:00:00


Running simulation:   0%|          | 8/3968 [00:00<04:58, 13.24days/s]

...

Running simulation:  28%|██▊       | 1124/3968 [02:14<06:52,  6.89days/s]

No data available for IDEXF on 2014-06-20 00:00:00


Running simulation:  28%|██▊       | 1129/3968 [02:14<06:49,  6.94days/s]

No data available for IDEXF on 2014-06-27 00:00:00


Running simulation: 100%|██████████| 3968/3968 [05:16<00:00, 12.54days/s]


In [57]:
# 4s na sucho
# 90s z backtesterem